In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, \
                            LSTM, LSTMCell, Embedding
from tensorflow_addons.seq2seq import BahdanauAttention, TrainingSampler, BasicDecoder
from keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from muse.Encodings import PianoRollEncoding, PerformanceEncoding

In [ ]:
input_encoding = PianoRollEncoding(sampling_frequency=4, normalize=True)

In [ ]:
output_encoding = PerformanceEncoding(time_unit=1/12,
                                      velocity_unit=16,
                                      use_velocity=True,
                                      use_all_off_event=True,
                                      use_drum_events=True,
                                      use_magenta=True)

In [ ]:
# normalize_velocity:
#   mean: 64.25399574283072
#   variance: 432.9165195560093
  # computed on the training set; ignored by default (unless run() is called with normalize_velocity=True)


In [ ]:
encoder_cnn = Sequential(name='encoder_cnn')
encoder_cnn.add(Conv2D(filters=32, 
                       kernel_size=(12, 12),
                       padding='same',
                       activation='elu'))
encoder_cnn.add(MaxPooling2D(pool_size=(2, 2),
                             strides=(2, 2)))
encoder_cnn.add(Conv2D(filters=32,
                       kernel_size=(4, 4),
                       padding='same',
                       activation='elu'))
encoder_cnn.add(MaxPooling2D(pool_size=(2, 4),
                             strides=(2, 4)))

In [ ]:
encoder_rnn = LSTM(units=200)

In [ ]:
style_encoder_cnn = Sequential(name='style_encoder_cnn')
style_encoder_cnn.add(Conv1D(filters=300,
                             kernel_size=6,
                             padding='same',
                             activation='elu'))
style_encoder_cnn.add(MaxPooling1D(pool_size=2,
                                   strides=2))
style_encoder_cnn.add(Conv1D(filters=300,
                             kernel_size=4,
                             padding='same',
                             activation='elu'))
style_encoder_cnn.add(MaxPooling1D(pool_size=2,
                                   strides=2))
style_encoder_cnn.add(Conv1D(filters=300,
                             kernel_size=4,
                             padding='same',
                             activation='elu'))
style_encoder_cnn.add(MaxPooling1D(pool_size=2,
                                   strides=2))

In [ ]:
style_encoder_rnn = LSTM(units=500)

In [ ]:
attention = BahdanauAttention(units=300)

In [ ]:
embedding = Embedding(output_dim=300)

In [ ]:
decoder_cell = LSTMCell(units=1024)
decoder = BasicDecoder(cell=decoder_cell,
                       sampler=TrainingSampler(),
                       output_layer=Dense(output_encoding.num_classes),
                       maximum_iterations=2000)

In [ ]:
lr_decay = ExponentialDecay(initial_learning_rate=0.001,
                            decay_steps=3000,
                            decay_rate=0.5)
optimizer = Adam(learning_rate=lr_decay)